In [9]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from src.config import get_parser
from src.util import make_faces
from pytorch3d.structures import Meshes

config = get_parser().parse_args(args=[])

blueprint = np.load(os.path.join(config.data_dir, config.blueprint)) 
points = torch.tensor(blueprint['points'])
normals = torch.tensor(blueprint['normals'])
points = F.interpolate(points, size=config.data_blueprint_size,
                               mode='bicubic', align_corners=True)
normals = F.interpolate(normals, size=config.data_blueprint_size, 
                        mode='bicubic', align_corners=True)   
points.shape, normals.shape

(torch.Size([1, 3, 640, 640]), torch.Size([1, 3, 640, 640]))

In [4]:
faces = torch.tensor(make_faces(config.data_blueprint_size, config.data_blueprint_size))
faces.shape

torch.Size([816642, 3])

In [7]:
bs = 5
verts = points.reshape(1, 3, -1).permute(0, 2, 1).expand(bs, -1, -1)
face_id = faces[None].expand(bs, -1, -1)
verts.shape, face_id.shape

(torch.Size([5, 409600, 3]), torch.Size([5, 816642, 3]))

In [10]:
trg_mesh = Meshes(verts=verts, faces=face_id)
trg_mesh

In [13]:
from src.loss.edge_loss import EdgeLoss

config.data_patch_size = config.data_blueprint_size
edgeLoss = EdgeLoss(config)
edgeLoss(verts)

tensor(8.8474e-05)